In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\Project Anchor\\MLflow-DVC-Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [57]:
%pwd

'c:\\Users\\User\\Desktop\\Project Anchor\\MLflow-DVC-Kidney-Disease-Classification'

In [55]:
# Update Training Entity
from dataclasses import dataclass
from pathlib import Path

In [94]:
@dataclass(frozen=True)
class TrainingConfig:
    """
    Configuration data class for training process.

    Attributes:
        root_dir (Path): Root directory for storing training artifacts.
        trained_model_path (Path): Path to save the trained model file.
        updated_base_model_path (Path): Path to the updated base model file.
        training_data (Path): Path to the directory or file containing training data.
        params_epochs (int): Number of epochs for training.
        params_batch_size (int): Batch size for training.
        params_is_augmentation (bool): Flag indicating whether data augmentation is enabled.
        params_image_size (list): Size of input images [width, height, channels].
    """
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [38]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [95]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # Path to the configuration file
        params_filepath = PARAMS_FILE_PATH    # Path to the parameters file
    ):

        # Read the configuration file and parameters file
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create necessary directories based on the configuration
        create_directories([self.config.artifacts_root])

    
    def get_training_config(self) -> TrainingConfig:
        # Extract training and preparation base model configurations from the main configuration
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model

        # Get parameters from the parameters file
        params = self.params

        # Construct the path to the training data based on the data ingestion configuration
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")

        # Create necessary directories for training artifacts
        create_directories([
            Path(training.root_dir)
        ])

        # Create a TrainingConfig object using the extracted configurations and parameters
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [96]:
# Update Component
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [97]:
class Training:
    def __init__(self, config: TrainingConfig):
        """
        Initializes the Training object with the provided configuration.

        Args:
            config (TrainingConfig): Configuration for the training process.
        """
        self.config = config

    
    def get_base_model(self):
        """
        Loads the updated base model specified in the configuration.

        This method loads the updated base model from the provided path and stores
        it in the `model` attribute of the Training object.
        """
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        """
        Prepares data generators for training and validation.

        This method initializes data generators for training and validation using
        the configuration parameters. It also applies data augmentation if specified
        in the configuration.
        """
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the trained model to the specified path.

        Args:
            path (Path): The path where the model will be saved.
            model (tf.keras.Model): The trained model to be saved.
        """
        model.save(path)


    
    def train(self):
        """
        Trains the model using the configured training data and parameters.

        This method trains the model using the specified training data and parameters
        such as epochs, batch size, etc., and saves the trained model to the specified
        path.
        """
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [98]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-05 22:56:28,748: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2024-03-05 22:56:28,752: INFO: common: YAML file 'params.yaml' loaded successfully]
[2024-03-05 22:56:28,755: INFO: common: Created directory at: artifacts]
[2024-03-05 22:56:28,757: INFO: common: Created directory at: artifacts\training]
Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
368/368 [==============================] - 1297s 4s/step - loss: 10.4527 - accuracy: 0.6232 - val_loss: 5.2195 - val_accuracy: 0.7136
